In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [12]:
%cd /content/drive/Shareddrives/CARD/projects/iNDI/Data/Discovery\ planning\ phase/Informatics_Lirong/projects_lirong/Florian_data

/content/drive/Shareddrives/CARD/projects/iNDI/Data/Discovery planning phase/Informatics_Lirong/projects_lirong/Florian_data


In [13]:
%ls -lth

total 14G
-rw------- 1 root root 2.6K Mar  5 01:28 4March2021_Florian_annotate.ipynb
-rw------- 1 root root  151 Jul 16  2020 Florian_data_June.gslides
-rw------- 1 root root 647M Jul 13  2020 cortical_ngn2_exclude_heg.h5ad
-rw------- 1 root root 728M Jul  8  2020 dopaminergic_exclude_heg.h5ad
-rw------- 1 root root 671M Jul  8  2020 cortical_exclude_heg.h5ad
drwx------ 2 root root 4.0K Jun 25  2020 write/
-rw------- 1 root root 1.2G Jun 23  2020 hypothalamic_exclude_heg.h5ad
drwx------ 2 root root 4.0K Jun 23  2020 data/
-rw------- 1 root root 9.2K Jun 23  2020 panglao_rank_genes_groups.csv
-rw------- 1 root root  165 Jun 17  2020 cluster.csv
-rw------- 1 root root  385 Jun 17  2020 panglao_cluster.csv
-rw------- 1 root root  359 Jun 17  2020 peng_cluster.csv
-rw------- 1 root root  949 Jun 17  2020 comparison_two_annotation.csv
-rw------- 1 root root  14K Jun 12  2020 hypothalamic_DE_0.6.csv
drwx------ 2 root root 4.0K Jun 12  2020 May2020_integrate_3_analysis/
drwx------ 2 root root

# analyze three conditions separatly

In [11]:
%ls -lth Florian_data/hypothalamic_sample_1/filtered_cellranger_matrix/

total 387M
-rw------- 1 root root 194M May 11  2020 matrix.mtx.gz
-rw------- 1 root root 298K May 11  2020 features.tsv.gz
-rw------- 1 root root  49K May 11  2020 barcodes.tsv.gz
-rw------- 1 root root 194M May  5  2020 hypothalamic_sample_1_filtered_cellranger_matrix.tar


In [14]:
PATH_cor = "cortical_dopaminergic_sample_1"
PATH_dop = "cortical_dopaminergic_sample_3"
PATH_hyp = "hypothalamic_sample_1"

In [18]:
%%bash -s {PATH_cor} {PATH_dop} {PATH_hyp}
for file in $1 $2 $3
do
echo "process ${file}"
ls ${file}/filtered_cellranger_matrix
done

process cortical_dopaminergic_sample_1
barcodes.tsv.gz
cortical_dopaminergic_sample_1_filtered_cellranger_matrix.tar
features.tsv.gz
matrix.mtx.gz
process cortical_dopaminergic_sample_3
barcodes.tsv.gz
cortical_dopaminergic_sample_3_filtered_cellranger_matrix.tar
features.tsv.gz
matrix.mtx.gz
process hypothalamic_sample_1
barcodes.tsv.gz
features.tsv.gz
hypothalamic_sample_1_filtered_cellranger_matrix.tar
matrix.mtx.gz


In [19]:
module_path = "/content/drive/MyDrive/scanpy_modules"

In [20]:
%ls -lth {module_path}

total 38K
drwx------ 2 root root 4.0K Mar  2 15:45 notebooks/
drwx------ 2 root root 4.0K Mar  2 15:13 notes/
drwx------ 2 root root 4.0K Mar  2 15:13 reference_markers/
-rw------- 1 root root 2.7K Feb 19 16:22 annotate_cluster_5.py
-rw------- 1 root root 4.7K Feb 19 16:22 cluster_2.py
-rw------- 1 root root 1.8K Feb 19 16:22 integrate.py
-rw------- 1 root root 4.5K Feb 19 16:22 marker_gene_3.py
-rw------- 1 root root 3.9K Feb 19 16:22 plot_markers_4.py
-rw------- 1 root root 7.1K Feb 19 16:22 qc_1.py


In [21]:
!pip install scanpy==1.5.1
!pip install leidenalg
!pip install umap-learn==0.4.6

     |████████████████████████████████| 7.2MB 5.4MB/s 
     |████████████████████████████████| 122kB 31.9MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 
     |████████████████████████████████| 2.4MB 5.7MB/s 
     |████████████████████████████████| 3.1MB 33.0MB/s 
     |████████████████████████████████| 71kB 3.1MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.4.6-cp37-none-any.whl size=67950 sha256=20a9f3bb4ce0b7eb8b97d93c5525c1b2e895f8b55b5a3ae75ac3c72ded286e7a
  Stored in directory: /root/.cache/pip/wheels/7d/1d/03/34aade9a9b97acddb8e93654eb856dadbf0964406eef8b96e2
Successfully built umap-learn
  Found existing installation: umap-learn 0.5.1
    Uninstalling umap-learn-0.5.1:
      Successfully uninstalled umap-learn-0.5.1


In [22]:
import h5py
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.precision", 2)

sc.settings.verbosity = 3
sc.logging.print_versions() # use in version 1.5
#sc.logging.print_header() # use in version 1.7
sc.settings.set_figure_params(dpi=80)

scanpy==1.5.1 anndata==0.7.5 umap==0.5.1 numpy==1.19.5 scipy==1.4.1 pandas==1.1.5 scikit-learn==0.22.2.post1 statsmodels==0.10.2 python-igraph==0.9.0 leidenalg==0.8.3


In [ ]:
# hypthalamics: 10000, 15
# cortical: 7500, 15
# dopaminergic:5500, 15

In [23]:
%%bash
for item in cortical dopaminergic hypothalamic
do
mkdir -p annotate/${item}
done

In [28]:
%cd annotate/cortical

/content/drive/Shareddrives/CARD/projects/iNDI/Data/Discovery planning phase/Informatics_Lirong/projects_lirong/Florian_data/annotate/cortical


In [34]:
%ls -lth {module_path}/notes/

total 26K
-rw------- 1 root root 2.2K Mar  4 15:28 3March2021_run_scanpy_iPSC.sh
-rw------- 1 root root 2.2K Mar  3 16:00 3March2021_run_scanpy_concat_NGN2_NIL_iPSC.sh
-rw------- 1 root root 2.2K Mar  3 16:00 4March2021_run_scanpy_Florian_cortical.sh
-rw------- 1 root root 2.2K Mar  3 16:00 4March2021_run_scanpy_Florian_dopaminergicsh.sh
-rw------- 1 root root 2.2K Mar  3 16:00 4March2021_run_scanpy_Florian_hypothalamic.sh
-rw------- 1 root root 2.2K Mar  3 15:39 2March2021_run_scanpy_concat_NGN2.sh
-rw------- 1 root root 2.2K Mar  3 15:38 3March2021_run_scanpy_concat_NIL.sh
drwx------ 2 root root 4.0K Mar  2 15:45 notes_brain_vs_ipsc/
drwx------ 2 root root 4.0K Mar  2 15:44 notes_SMAD/


In [41]:
%ls -lth  ../../{PATH_cor}/

total 4.9M
drwx------ 2 root root 4.0K May 11  2020 filtered_cellranger_matrix/
drwx------ 2 root root 4.0K May  7  2020 cellSNP/
drwx------ 2 root root 4.0K May  6  2020 cache/
-rw------- 1 root root 4.9M May  6  2020 scanpy_florian_cortical_doma_1.ipynb
drwx------ 2 root root 4.0K May  6  2020 figures/
drwx------ 2 root root 4.0K May  5  2020 cellSNP_out/
drwx------ 2 root root 4.0K May  5  2020 scanpy_out/
drwx------ 2 root root 4.0K May  5  2020 vireo_out/


# Process the cortical_1

In [43]:
RUN_SH="4March2021_run_scanpy_Florian_cortical.sh"
!bash {module_path}/notes/{RUN_SH} ../../{PATH_cor}/filtered_cellranger_matrix

Processing ../../cortical_dopaminergic_sample_1/filtered_cellranger_matrix
The arguments are:  Namespace(color_gene='MAP2', dpi=80, exclude_highly_expressed=True, figsize=None, figure_type='pdf', input='../../cortical_dopaminergic_sample_1/filtered_cellranger_matrix', min_cells=3, min_genes=200, n_genes_by_counts=7000, n_top=20, out='count_after_QC', pct_counts_mt=20.0, project='', show=False)

exclude_highly_expressed is  True

scanpy==1.5.1 anndata==0.7.5 umap==0.4.6 numpy==1.19.5 scipy==1.4.1 pandas==1.1.5 scikit-learn==0.22.2.post1 statsmodels==0.10.2 python-igraph==0.9.0 leidenalg==0.8.3
Running Scanpy 1.5.1, on 2021-03-05 01:59.
... writing an h5ad cache file to speedup reading next time
normalizing counts per cell
    finished (0:00:00)
filtered out 12489 genes that are detected in less than 3 cells
... storing 'feature_types' as categorical
/usr/local/lib/python3.7/dist-packages/seaborn/_core.py:1319: UserWarning: Vertical orientation ignored with only `x` specified.
  warnings

In [44]:
import os
marker_path="reference_markers/marker_panglao_dic.p"
marker_file = os.path.join(module_path,marker_path)
print(os.path.join(module_path,marker_path))

import pickle

with open(marker_file, "rb") as fd:
    marker_ref = pickle.load(fd)

/content/drive/MyDrive/scanpy_modules/reference_markers/marker_panglao_dic.p


In [45]:
adata=sc.read_h5ad("after_ranking_gene.h5ad")
adata

AnnData object with n_obs × n_vars = 10613 × 3375
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'leiden', 'leiden_0.2_colors', 'leiden_0.4_colors', 'leiden_0.6_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'rank_genes_groups_r0.2', 'rank_genes_groups_r0.4', 'rank_genes_groups_r0.6', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [47]:
# generate the annotation for each comparison method
def cell_types(adata, marker=marker_ref, key=None, method="overlap_count", normalize=None):
  """
  Draw overlap matrix using different methods according to the marker dictionary
  """
  annotation = sc.tl.marker_gene_overlap(adata, marker, key=key, method=method, normalize=normalize)

  # repress the plotting here
  # sns.set(rc={'figure.figsize':(12, 12)})
  # sns.heatmap(annotation, annot=True)

  cell_types = []
  # choosing the candidate with the highest value as the cell type
  for item in annotation.columns.to_list():
    cell_type = annotation[item].idxmax()
    cell_types.append(cell_type)

  return cell_types


# generate a dataframe to compare the annotation results from 3 methods for a specfic resolution
def calculate_types(adata, marker=marker_ref, key="rank_genes_groups"):
    type_1 = cell_types(adata, marker=marker_ref, key=key, method="overlap_count", normalize="data")
    type_2 = cell_types(adata, marker=marker_ref, key=key, method="jaccard")
    type_3 = cell_types(adata, marker=marker_ref, key=key, method="overlap_coef")
    types = pd.DataFrame({"type1": type_1, "type2": type_2, "type3": type_3})
    
    return types


# add annotation info to adata using the "best" one
def define_types(adata, types, leiden_r="leiden_0.4"):
    adata=adata
    types=types
    cell_types = types["type3"].to_dict()
    cell_types_str = {str(key): val for key, val in cell_types.items()}
    adata.obs[leiden_r+"_annotation"] = adata.obs[leiden_r].replace(cell_types_str)
    print(adata.obs[leiden_r+"_annotation"].value_counts())

In [48]:
for r in ["0.2", "0.4", "0.6"]:
  the_type = calculate_types(adata, key="rank_genes_groups_r"+r)
  print("the cell types on the resolutions of ", r)
  define_types(adata, the_type, leiden_r="leiden_"+r)
  print()

the cell types on the resolutions of  0.2
Bergmann glia                  5710
Adrenergic neurons             1657
Neural stem/precursor cells    1040
Glutaminergic neurons           858
Pyramidal cells                 798
Tanycytes                       550
Name: leiden_0.2_annotation, dtype: int64

the cell types on the resolutions of  0.4
Tanycytes                      2459
Adrenergic neurons             2432
Glutaminergic neurons          2255
Bergmann glia                  2109
Pyramidal cells                1219
Neural stem/precursor cells     139
Name: leiden_0.4_annotation, dtype: int64

the cell types on the resolutions of  0.6
Adrenergic neurons             3409
Tanycytes                      2548
Glutaminergic neurons          1494
Radial glia cells              1209
Satellite glial cells          1208
Bergmann glia                   571
Neural stem/precursor cells     174
Name: leiden_0.6_annotation, dtype: int64



In [49]:
adata.write_h5ad("after_annotate.h5ad")
adata

... storing 'leiden_0.2_annotation' as categorical
... storing 'leiden_0.4_annotation' as categorical
... storing 'leiden_0.6_annotation' as categorical


AnnData object with n_obs × n_vars = 10613 × 3375
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6', 'leiden_0.2_annotation', 'leiden_0.4_annotation', 'leiden_0.6_annotation'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'leiden', 'leiden_0.2_colors', 'leiden_0.4_colors', 'leiden_0.6_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'rank_genes_groups_r0.2', 'rank_genes_groups_r0.4', 'rank_genes_groups_r0.6', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

# Process dopaminergic_1

In [76]:
%cd ../dopaminergic/

/content/drive/Shareddrives/CARD/projects/iNDI/Data/Discovery planning phase/Informatics_Lirong/projects_lirong/Florian_data/annotate/dopaminergic


In [78]:
% ls -lth

total 1.1G
drwx------ 2 root root 4.0K Mar  5 02:47 figures/
-rw------- 1 root root 351M Mar  5 02:46 after_ranking_gene.h5ad
-rw------- 1 root root  22K Mar  5 02:46 rank_genes_groups_r0.6.csv
-rw------- 1 root root  16K Mar  5 02:45 rank_genes_groups_r0.4.csv
-rw------- 1 root root  11K Mar  5 02:44 rank_genes_groups_r0.2.csv
-rw------- 1 root root  34K Mar  5 02:44 rank_genes_groups.csv
-rw------- 1 root root 351M Mar  5 02:43 after_leiden.h5ad
-rw------- 1 root root 348M Mar  5 02:38 count_after_QC.h5ad
drwx------ 2 root root 4.0K Mar  5 02:37 cache/


In [77]:
%ls -lth {module_path}/notes/

total 26K
-rw------- 1 root root 2.2K Mar  4 15:28 3March2021_run_scanpy_iPSC.sh
-rw------- 1 root root 2.2K Mar  3 16:00 3March2021_run_scanpy_concat_NGN2_NIL_iPSC.sh
-rw------- 1 root root 2.2K Mar  3 16:00 4March2021_run_scanpy_Florian_cortical.sh
-rw------- 1 root root 2.2K Mar  3 16:00 4March2021_run_scanpy_Florian_dopaminergicsh.sh
-rw------- 1 root root 2.2K Mar  3 16:00 4March2021_run_scanpy_Florian_hypothalamic.sh
-rw------- 1 root root 2.2K Mar  3 15:39 2March2021_run_scanpy_concat_NGN2.sh
-rw------- 1 root root 2.2K Mar  3 15:38 3March2021_run_scanpy_concat_NIL.sh
drwx------ 2 root root 4.0K Mar  2 15:45 notes_brain_vs_ipsc/
drwx------ 2 root root 4.0K Mar  2 15:44 notes_SMAD/


In [62]:
RUN_SH="4March2021_run_scanpy_Florian_dopaminergicsh.sh"
!bash {module_path}/notes/{RUN_SH} ../../{PATH_dop}/filtered_cellranger_matrix

Processing ../../cortical_dopaminergic_sample_3/filtered_cellranger_matrix
The arguments are:  Namespace(color_gene='MAP2', dpi=80, exclude_highly_expressed=True, figsize=None, figure_type='pdf', input='../../cortical_dopaminergic_sample_3/filtered_cellranger_matrix', min_cells=3, min_genes=200, n_genes_by_counts=7000, n_top=20, out='count_after_QC', pct_counts_mt=20.0, project='', show=False)

exclude_highly_expressed is  True

scanpy==1.5.1 anndata==0.7.5 umap==0.4.6 numpy==1.19.5 scipy==1.4.1 pandas==1.1.5 scikit-learn==0.22.2.post1 statsmodels==0.10.2 python-igraph==0.9.0 leidenalg==0.8.3
Running Scanpy 1.5.1, on 2021-03-05 02:35.
... writing an h5ad cache file to speedup reading next time
normalizing counts per cell
    finished (0:00:00)
filtered out 11728 genes that are detected in less than 3 cells
... storing 'feature_types' as categorical
/usr/local/lib/python3.7/dist-packages/seaborn/_core.py:1319: UserWarning: Vertical orientation ignored with only `x` specified.
  warnings

In [68]:
%ls -lth

total 1.1G
drwx------ 2 root root 4.0K Mar  5 02:47 figures/
-rw------- 1 root root 351M Mar  5 02:46 after_ranking_gene.h5ad
-rw------- 1 root root  22K Mar  5 02:46 rank_genes_groups_r0.6.csv
-rw------- 1 root root  16K Mar  5 02:45 rank_genes_groups_r0.4.csv
-rw------- 1 root root  11K Mar  5 02:44 rank_genes_groups_r0.2.csv
-rw------- 1 root root  34K Mar  5 02:44 rank_genes_groups.csv
-rw------- 1 root root 351M Mar  5 02:43 after_leiden.h5ad
-rw------- 1 root root 348M Mar  5 02:38 count_after_QC.h5ad
drwx------ 2 root root 4.0K Mar  5 02:37 cache/


In [79]:
adata = sc.read_h5ad("after_ranking_gene.h5ad")
adata

AnnData object with n_obs × n_vars = 9792 × 3598
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'leiden', 'leiden_0.2_colors', 'leiden_0.4_colors', 'leiden_0.6_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'rank_genes_groups_r0.2', 'rank_genes_groups_r0.4', 'rank_genes_groups_r0.6', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [80]:
for r in ["0.2", "0.4", "0.6"]:
  the_type = calculate_types(adata, key="rank_genes_groups_r"+r)
  print("the cell types on the resolutions of ", r)
  define_types(adata, the_type, leiden_r="leiden_"+r)
  print()

the cell types on the resolutions of  0.2
Meningeal cells                3937
Adrenergic neurons             2546
Radial glia cells              2059
Neural stem/precursor cells    1089
Glutaminergic neurons           101
Schwann cells                    60
Name: leiden_0.2_annotation, dtype: int64

the cell types on the resolutions of  0.4
Meningeal cells                3587
Adrenergic neurons             2234
Radial glia cells              2019
Dopaminergic neurons            933
Neural stem/precursor cells     863
Glutaminergic neurons            96
Schwann cells                    60
Name: leiden_0.4_annotation, dtype: int64

the cell types on the resolutions of  0.6
Meningeal cells                3540
Adrenergic neurons             2166
Neural stem/precursor cells    1269
Radial glia cells              1149
Dopaminergic neurons            958
Tanycytes                       548
Glutaminergic neurons           102
Schwann cells                    60
Name: leiden_0.6_annotation, dty

In [81]:
adata.write_h5ad("after_annotate.h5ad")
adata


... storing 'leiden_0.2_annotation' as categorical
... storing 'leiden_0.4_annotation' as categorical
... storing 'leiden_0.6_annotation' as categorical


AnnData object with n_obs × n_vars = 9792 × 3598
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6', 'leiden_0.2_annotation', 'leiden_0.4_annotation', 'leiden_0.6_annotation'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'leiden', 'leiden_0.2_colors', 'leiden_0.4_colors', 'leiden_0.6_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'rank_genes_groups_r0.2', 'rank_genes_groups_r0.4', 'rank_genes_groups_r0.6', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

# Process the hypothalamic_1

In [69]:
%cd ../hypothalamic/

/content/drive/Shareddrives/CARD/projects/iNDI/Data/Discovery planning phase/Informatics_Lirong/projects_lirong/Florian_data/annotate/hypothalamic


In [ ]:
PATH_cor = "cortical_dopaminergic_sample_1"
PATH_dop = "cortical_dopaminergic_sample_3"
PATH_hyp = "hypothalamic_sample_1"

In [70]:
%ls -lth

total 0


In [71]:
RUN_SH="4March2021_run_scanpy_Florian_hypothalamic.sh"
!bash {module_path}/notes/{RUN_SH} ../../{PATH_hyp}/filtered_cellranger_matrix

Processing ../../hypothalamic_sample_1/filtered_cellranger_matrix
The arguments are:  Namespace(color_gene='MAP2', dpi=80, exclude_highly_expressed=True, figsize=None, figure_type='pdf', input='../../hypothalamic_sample_1/filtered_cellranger_matrix', min_cells=3, min_genes=200, n_genes_by_counts=7000, n_top=20, out='count_after_QC', pct_counts_mt=20.0, project='', show=False)

exclude_highly_expressed is  True

scanpy==1.5.1 anndata==0.7.5 umap==0.4.6 numpy==1.19.5 scipy==1.4.1 pandas==1.1.5 scikit-learn==0.22.2.post1 statsmodels==0.10.2 python-igraph==0.9.0 leidenalg==0.8.3
Running Scanpy 1.5.1, on 2021-03-05 02:57.
... writing an h5ad cache file to speedup reading next time
normalizing counts per cell
    finished (0:00:00)
filtered out 8 cells that have less than 200 genes expressed
filtered out 9029 genes that are detected in less than 3 cells
... storing 'feature_types' as categorical
/usr/local/lib/python3.7/dist-packages/seaborn/_core.py:1319: UserWarning: Vertical orientation i

In [72]:
%ls -lth

total 1.3G
drwx------ 2 root root 4.0K Mar  5 03:10 figures/
-rw------- 1 root root 412M Mar  5 03:09 after_ranking_gene.h5ad
-rw------- 1 root root  23K Mar  5 03:09 rank_genes_groups_r0.6.csv
-rw------- 1 root root  19K Mar  5 03:08 rank_genes_groups_r0.4.csv
-rw------- 1 root root 8.5K Mar  5 03:07 rank_genes_groups_r0.2.csv
-rw------- 1 root root  30K Mar  5 03:06 rank_genes_groups.csv
-rw------- 1 root root 412M Mar  5 03:05 after_leiden.h5ad
-rw------- 1 root root 410M Mar  5 03:02 count_after_QC.h5ad
drwx------ 2 root root 4.0K Mar  5 03:00 cache/


In [73]:
adata = sc.read_h5ad("after_ranking_gene.h5ad")
adata

AnnData object with n_obs × n_vars = 6848 × 5567
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'leiden', 'leiden_0.2_colors', 'leiden_0.4_colors', 'leiden_0.6_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'rank_genes_groups_r0.2', 'rank_genes_groups_r0.4', 'rank_genes_groups_r0.6', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [74]:
for r in ["0.2", "0.4", "0.6"]:
  the_type = calculate_types(adata, key="rank_genes_groups_r"+r)
  print("the cell types on the resolutions of ", r)
  define_types(adata, the_type, leiden_r="leiden_"+r)
  print()

the cell types on the resolutions of  0.2
Tanycytes               2132
Neuroblasts             1719
Adrenergic neurons      1336
Radial glia cells       1280
Bergmann glia            249
Dopaminergic neurons     132
Name: leiden_0.2_annotation, dtype: int64

the cell types on the resolutions of  0.4
Tanycytes                      1636
Radial glia cells              1142
Neurons                         894
Glycinergic neurons             792
Cholinergic neurons             653
Bergmann glia                   572
Adrenergic neurons              361
Neuroblasts                     269
Neural stem/precursor cells     225
Meningeal cells                 166
Glutaminergic neurons           138
Name: leiden_0.4_annotation, dtype: int64

the cell types on the resolutions of  0.6
Tanycytes                      1628
Radial glia cells              1143
Neurons                         614
Bergmann glia                   574
Pyramidal cells                 566
Glycinergic neurons             498
Ch

In [75]:
adata.write_h5ad("after_annotate.h5ad")
adata

... storing 'leiden_0.2_annotation' as categorical
... storing 'leiden_0.4_annotation' as categorical
... storing 'leiden_0.6_annotation' as categorical


AnnData object with n_obs × n_vars = 6848 × 5567
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_0.2', 'leiden_0.4', 'leiden_0.6', 'leiden_0.2_annotation', 'leiden_0.4_annotation', 'leiden_0.6_annotation'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'leiden', 'leiden_0.2_colors', 'leiden_0.4_colors', 'leiden_0.6_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'rank_genes_groups_r0.2', 'rank_genes_groups_r0.4', 'rank_genes_groups_r0.6', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'